# This file has all the cool plotting code.

In [113]:
import os
import pandas as pd
import re
from rich.console import Console
from rich.table import Table
import math

In [114]:
console = Console()

In [115]:
outfile_loc = "output"

In [116]:
heapdfs = {}
quickdfs = {}

In [133]:
for _, _, files in os.walk(outfile_loc):
    for file in files:

        size = int(re.findall(r'size(\d+)', file)[0])
        manip_type = re.findall(r'\d_(\w+).txt', file)[0]

        if "_heap_" in file:
            if size not in heapdfs:
                heapdfs[size] = {}
                heapdfs[size][manip_type] = pd.read_csv(outfile_loc + "/" + file)
            else:
                heapdfs[size][manip_type] = pd.read_csv(outfile_loc + "/" + file)

        elif "_quick_" in file:
            
            if size not in quickdfs:
                quickdfs[size] = {}
                quickdfs[size][manip_type] = pd.read_csv(outfile_loc + "/" + file)
            else:
                quickdfs[size][manip_type] = pd.read_csv(outfile_loc + "/" + file)
                

In [134]:
len(heapdfs), len(quickdfs)

(4, 4)

In [135]:
def pretty_print_data(dfs, size):

    table = Table(title=f"Size {size} | n log n: {round(size * math.log(size, 2), 4)} | 1.5 n log n: {round(size * 1.5 * math.log(size, 2), 4)}")
    table.add_column("")
    table.add_column("Sorted")
    table.add_column("Reverse")
    table.add_column("Random Shuffle")
    table.add_row("Comparisons", str(dfs[size]['sorted']["comparison_count"].mean()), str(dfs[size]['reverse']["comparison_count"].mean()), str(dfs[size]['shuffle']["comparison_count"].mean()))
    table.add_row("Time", str(dfs[size]['sorted']["time"].mean()), str(dfs[size]['reverse']["time"].mean()), str(dfs[size]['shuffle']["time"].mean()))

    console.print(table)

In [136]:
print("HEAP")
pretty_print_data(heapdfs, 10)
pretty_print_data(heapdfs, 100)
pretty_print_data(heapdfs, 1000)
pretty_print_data(heapdfs, 10000)

HEAP


 Size 10 | n log n: 33.2193 | 1.5 n log n: 49.8289  
┏━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃             ┃ Sorted  ┃ Reverse ┃ Random Shuffle ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Comparisons │ 42.0    │ 29.0    │ 37.053         │
│ Time        │ 647.783 │ 527.506 │ 799.833        │
└─────────────┴─────────┴─────────┴────────────────┘

  Size 100 | n log n: 664.3856 | 1.5 n log n: 996.5784  
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃             ┃ Sorted    ┃ Reverse   ┃ Random Shuffle ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Comparisons │ 954.0     │ 745.0     │ 850.639        │
│ Time        │ 17439.123 │ 14668.874 │ 17756.709      │
└─────────────┴───────────┴───────────┴────────────────┘

Size 1000 | n log n: 9965.7843 | 1.5 n log n: 14948.6764 
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃             ┃ Sorted    ┃ Reverse    ┃ Random Shuffle ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Comparisons │ 14632.988 │ 12170.0    │ 13484.568      │
│ Time        │ 255209.29 │ 235174.217 │ 268415.58      │
└─────────────┴───────────┴────────────┴────────────────┘

Size 10000 | n log n: 132877.1238 | 1.5 n log n: 199315.6857
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃             ┃ Sorted      ┃ Reverse     ┃ Random Shuffle ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Comparisons │ 196227.027  │ 171953.701  │ 184434.863     │
│ Time        │ 3376442.951 │ 3154233.431 │ 4636465.839    │
└─────────────┴─────────────┴─────────────┴────────────────┘

In [137]:
print("QUICK")
pretty_print_data(quickdfs, 10)
pretty_print_data(quickdfs, 100)
pretty_print_data(quickdfs, 1000)
pretty_print_data(quickdfs, 10000)

QUICK


 Size 10 | n log n: 33.2193 | 1.5 n log n: 49.8289 
┏━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃             ┃ Sorted ┃ Reverse ┃ Random Shuffle ┃
┡━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Comparisons │ 29.0   │ 49.0    │ 28.42          │
│ Time        │ 516.21 │ 710.514 │ 717.614        │
└─────────────┴────────┴─────────┴────────────────┘

  Size 100 | n log n: 664.3856 | 1.5 n log n: 996.5784  
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃             ┃ Sorted    ┃ Reverse   ┃ Random Shuffle ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Comparisons │ 2549.0    │ 4999.0    │ 694.842        │
│ Time        │ 22608.041 │ 36715.609 │ 13605.367      │
└─────────────┴───────────┴───────────┴────────────────┘

  Size 1000 | n log n: 9965.7843 | 1.5 n log n: 14948.6764  
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃             ┃ Sorted      ┃ Reverse     ┃ Random Shuffle ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Comparisons │ 250499.414  │ 499997.692  │ 11496.98       │
│ Time        │ 1948921.775 │ 3231777.214 │ 198635.583     │
└─────────────┴─────────────┴─────────────┴────────────────┘

  Size 10000 | n log n: 132877.1238 | 1.5 n log n: 199315.6857  
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃             ┃ Sorted        ┃ Reverse       ┃ Random Shuffle ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Comparisons │ 25004925.819  │ 49998684.722  │ 160695.07      │
│ Time        │ 189666059.368 │ 319570741.156 │ 2741025.421    │
└─────────────┴───────────────┴───────────────┴────────────────┘